5-7. 미니 프로젝트 : 가위바위보 분류기를 만들자 


<<목차>>

1)데이터를 준비하자

2)딥러닝 네트워크 설계하기

3)딥러닝 네트워크 학습시키기

4)얼마나 잘 만들었는지 확인하기(테스트)

5)더 좋은 네트워크 만들어보기

6)루브릭

7)회고

1)데이터를 준비하자

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import glob
import os
from tensorflow import keras
import matplotlib.pyplot as plt

print(tf.__version__)
print(np.__version__)

2022-10-07 23:31:13.240545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 23:31:13.442685: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 23:31:14.012099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.5/lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2022-10-07 23:31:14.012156: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not l

2.10.0
1.23.3


In [2]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/rock"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [3]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/scissor"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [4]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/paper"
resize_images(image_dir_path)

94  images to be resized.
94  images resized.


이미지들을 모두 28x28으로 리사이즈

In [5]:
import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

2)딥러닝 네트워크 설계하기

In [6]:
image_dir_path = "/home/june/다운로드/RCP"
(x_train, y_train)=load_data(image_dir_path, 2400)

x_train_norm = x_train/255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 294 입니다.
x_train shape: (2400, 28, 28, 3)
y_train shape: (2400,)


In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(256, (3, 3), activation = 'relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 512)        0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 3, 3, 512)         2359808   
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 1, 1, 512)       

3)딥러닝 네트워크 학습시키기

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
    
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


2022-10-07 23:29:26.959463: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:29:26.959663: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:29:26.959933: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:29:26.960156: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:29:36.959886: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:29:36.960375: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (2

KeyboardInterrupt: 

4)얼마나 잘 만들었는지 확인하기(테스트)

In [ ]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/rock"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [ ]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/scissor"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [ ]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/paper"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [ ]:
image_dir_path = "/home/june/다운로드/RCP/test"
(x_test, y_test)=load_data(image_dir_path, 300)

x_test_norm = x_test/255.0

학습데이터(x_train)의 이미지 개수는 300 입니다.


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

2022-10-07 23:30:09.822174: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:30:09.822458: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:30:19.823137: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:30:19.823789: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 22.41M (23494656 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-10-07 23:30:19.823835: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 21.12MiB (rounded to 22151168)requested by op sequential_2/conv2d_6/Relu
If the cause is memory fragmentation maybe the environment variable 'TF_G

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_2/conv2d_6/Relu' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/june/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/june/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 978, in launch_instance
      app.start()
    File "/home/june/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/june/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    Fil

6) 루브릭


1. 이미지 분류기 모델이 성공적으로 만들어졌는가?

학습과정이 정상적으로 수행되었으며, 학습 결과에 대한 그래프를 시각화(ex. train acc / train loss / val acc / val loss 등) 해 보았음

2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가?

오버피팅 극복을 위하여 데이터셋의 다양성, 정규화 등을 2가지 이상 시도해보았음

3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?

60% 이상 도달하였음
